In [2]:
import sys
sys.path.append('../utils/')
from parsing import *

import warnings
warnings.filterwarnings("ignore")

PATH_INTERMEDIATE = '../../data/intermediate/'
PATH_RAW = '../../data/raw/'

## OZP

In [43]:
insurance_companies = pd.DataFrame()

In [3]:
ozp = pd.read_excel(PATH_RAW+'ic/ozp.xlsx')

In [44]:
ozp.columns = ['year', 'facilities_id', 'expertise', 'count'] # rok, identifikacni cislo strediska, odbornost, pocet vykonu
ozp['insurance_company'] = 'OZP'
ozp.info()
insurance_companies = insurance_companies.append(ozp)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343327 entries, 0 to 343326
Data columns (total 5 columns):
year                 343327 non-null int64
facilities_id        343327 non-null int64
expertise            343325 non-null object
count                343327 non-null int64
insurance_company    343327 non-null object
dtypes: int64(3), object(2)
memory usage: 13.1+ MB


In [10]:
vzp = pd.read_excel(PATH_RAW+'ic/vzp.xlsx')

In [45]:
vzp.columns = ['year', 'expertise', 'procedure_code', 'region', 'count'] # rok, odbornost, kod vykonu, region, pocet vykonu
vzp['insurance_company'] = 'VZP'
vzp.info()
insurance_companies = insurance_companies.append(vzp)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455390 entries, 0 to 455389
Data columns (total 6 columns):
year                 455390 non-null int64
expertise            455390 non-null object
procedure_code       455390 non-null int64
region               455390 non-null object
count                455390 non-null int64
insurance_company    455390 non-null object
dtypes: int64(3), object(3)
memory usage: 20.8+ MB


## SKODA

In [55]:
skoda = pd.read_csv(PATH_RAW+'ic/skoda.csv', sep=';', usecols=['odb','sum_vykony','ico','rok'])

In [56]:
skoda.columns = ['expertise', 'count', 'facilities_id', 'year'] # rok, odbornost, kod vykonu, region, pocet vykonu
skoda['insurance_company'] = 'SKODA'
skoda.info()
insurance_companies = insurance_companies.append(skoda)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80502 entries, 0 to 80501
Data columns (total 5 columns):
expertise            80491 non-null object
count                80502 non-null int64
facilities_id        80435 non-null float64
year                 80502 non-null int64
insurance_company    80502 non-null object
dtypes: float64(1), int64(2), object(2)
memory usage: 3.1+ MB


In [70]:
insurance_companies.reset_index().to_feather(PATH_INTERMEDIATE+'insurance_companies.ftr')
insurance_companies.info()

# https://szv.mzcr.cz/Ciselnik/Odbornost?page=4

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879219 entries, 0 to 80501
Data columns (total 7 columns):
count                879219 non-null int64
expertise            879206 non-null object
facilities_id        423762 non-null float64
insurance_company    879219 non-null object
procedure_code       455390 non-null float64
region               455390 non-null object
year                 879219 non-null int64
dtypes: float64(2), int64(2), object(3)
memory usage: 53.7+ MB


## NRPZS

In [78]:
nrpzs = pd.read_csv(PATH_RAW+'nrpzs.csv')
nrpzs.columns

Index(['MistoPoskytovaniId', 'ZdravotnickeZarizeniId', 'Kod', 'NazevZarizeni',
       'DruhZarizeni', 'Obec', 'Psc', 'Ulice', 'CisloDomovniOrientacni',
       'Kraj', 'KrajCode', 'Okres', 'OkresCode', 'SpravniObvod',
       'PoskytovatelTelefon', 'PoskytovatelFax', 'PoskytovatelEmail',
       'PoskytovatelWeb', 'Ico', 'TypOsoby', 'PravniFormaKod', 'KrajCodeSidlo',
       'OkresCodeSidlo', 'ObecSidlo', 'PscSidlo', 'UliceSidlo',
       'CisloDomovniOrientacniSidlo', 'OborPece', 'FormaPece', 'DruhPece',
       'OdbornyZastupce', 'Lat', 'Lng'],
      dtype='object')

In [118]:
nrpzs = nrpzs[[x for x in nrpzs.columns if not 'Poskytovatel' in x and not 'Code' in x and not 'Kod' in x]]
display(nrpzs.columns)

nrpzs.head()

Index(['MistoPoskytovaniId', 'ZdravotnickeZarizeniId', 'NazevZarizeni',
       'DruhZarizeni', 'Obec', 'Psc', 'Ulice', 'CisloDomovniOrientacni',
       'Kraj', 'Okres', 'SpravniObvod', 'Ico', 'TypOsoby', 'ObecSidlo',
       'PscSidlo', 'UliceSidlo', 'CisloDomovniOrientacniSidlo', 'OborPece',
       'FormaPece', 'DruhPece', 'OdbornyZastupce', 'Lat', 'Lng'],
      dtype='object')

,MistoPoskytovaniId,ZdravotnickeZarizeniId,NazevZarizeni,DruhZarizeni,Obec,Psc,Ulice,CisloDomovniOrientacni,Kraj,Okres,...,ObecSidlo,PscSidlo,UliceSidlo,CisloDomovniOrientacniSidlo,OborPece,FormaPece,DruhPece,OdbornyZastupce,Lat,Lng
0,240269,161751,Mgr. Kateřina Konečná,Samostatné zařízení fyzioterapeuta,Olomouc,77900.0,Březinova,136/7,Olomoucký kraj,Olomouc,...,NaN,NaN,NaN,NaN,Fyzioterapeut,zdrav. péče poskytovaná ve vlastním soc. prost...,léčebně rehabilitační péče,NaN,49.592352,17.273803
1,240267,161750,VAŠE ZUBNÍ CENTRUM s.r.o.,Samostatná ordinace PL - stomatologa,Louny,44001.0,Kosmonautů,2304,Ústecký kraj,Louny,...,Ústí nad Labem,40001.0,Masarykova,118/147,zubní lékařství,primární ambulantní péče,NaN,MUDr. VĚRA NOVÁČKOVÁ,50.358212,13.791025
2,240243,161747,Opti Medical s.r.o.,Oční optika,Vysoké Mýto,56601.0,nám. Přemysla Otakara II.,13,Pardubický kraj,Ústí nad Orlicí,...,Pardubice,53002.0,Havlíčkova,445,Optometrista,ambulantní péče,NaN,Adéla Schejbalová,49.954398,16.160298
3,240260,161746,K i S MAX s.r.o.,Samostatná ordinace PL - stomatologa,Chýnov,39155.0,Nádražní,400,Jihočeský kraj,Tábor,...,Chýnov,39155.0,Nádražní,400,zubní lékařství,ambulantní péče,NaN,Petr Chrášťanský,49.414237,14.811757
4,240254,161745,Magnolia Medical s.r.o.,Výdejna zdravotnických prostředků,Karlovy Vary,36005.0,Sokolovská,138/74,Karlovarský kraj,Karlovy Vary,...,Teplička,36464.0,NaN,6,NaN,NaN,lékárenská péče,MARTINA ŠTELLNEROVÁ,50.234089,12.857118


In [135]:
display(nrpzs.groupby('Obec').count()['MistoPoskytovaniId'].sort_values().tail(5))
nrpzs.groupby('Okres').count()['MistoPoskytovaniId'].sort_values().tail(20)

Obec
Praha 5     661
Praha 4     766
Plzeň       897
Ostrava    1250
Brno       2101
Name: MistoPoskytovaniId, dtype: int64

Okres
Ústí nad Orlicí        474
Nový Jičín             494
Hodonín                519
Vsetín                 525
Uherské Hradiště       554
Liberec                571
Opava                  572
Brno-venkov            583
Pardubice              601
Karlovy Vary           647
Hradec Králové         662
Frýdek-Místek          705
Karviná                747
České Budějovice       747
Zlín                   837
Plzeň-město            928
Olomouc               1067
Ostrava-město         1335
Brno-město            2101
Hlavní město Praha    6216
Name: MistoPoskytovaniId, dtype: int64